In [1]:
import fastf1 as f1
import fastf1.plotting
import matplotlib.pyplot as plt
import numpy as np
fastf1.plotting.setup_mpl()
import datetime
import os

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.io import show
from fastf1.ergast import Ergast


fastf1.Cache.enable_cache(os.getcwd())

In [2]:
current_race_idx=fastf1.get_events_remaining()['RoundNumber'].keys()[0]

In [3]:
eg = Ergast()
sch = eg.get_race_schedule(2024)
results=[]

for round, race in sch['raceName'].items():
    #print(f"{round+1} {race}")
    if round+1<current_race_idx:
        # Extract results
        temp = eg.get_race_results(2024, round=round+1)
        if temp.content:
            temp=temp.content[0]

            # Add sprint if happens
            sprint = eg.get_sprint_results(2024, round=round+1)
            if sprint.content and sprint.description['round'][0] == round+1:
                temp = pd.merge(temp, sprint.content[0], on='driverCode', how='left')
                temp['points'] = temp['points_x'] + temp['points_y']
                temp.drop(columns=['points_x', 'points_y'], inplace=True)

            # Rd # and Name
            temp['round']=round+1
            temp['race']=race.removesuffix(' Grand Prix')
            temp=temp[['round','race', 'driverCode', 'points']]
            results.append(temp)
results=pd.concat(results)
races = results['race'].drop_duplicates()
print(results)

    round     race driverCode  points
0       1  Bahrain        VER    26.0
1       1  Bahrain        PER    18.0
2       1  Bahrain        SAI    15.0
3       1  Bahrain        LEC    12.0
4       1  Bahrain        RUS    10.0
..    ...      ...        ...     ...
15      8   Monaco        ZHO     0.0
16      8   Monaco        OCO     0.0
17      8   Monaco        PER     0.0
18      8   Monaco        HUL     0.0
19      8   Monaco        MAG     0.0

[159 rows x 4 columns]


In [4]:
points_per_driver = results.groupby('driverCode')['points'].sum()

# Print the result
print(points_per_driver)

driverCode
ALB      2.0
ALO     33.0
BEA      6.0
BOT      0.0
GAS      1.0
HAM     42.0
HUL      6.0
LEC    138.0
MAG      1.0
NOR    113.0
OCO      1.0
PER    107.0
PIA     71.0
RIC      5.0
RUS     54.0
SAI    108.0
SAR      0.0
STR     11.0
TSU     19.0
VER    169.0
ZHO      0.0
Name: points, dtype: float64


In [5]:
# Rearrange
results = results.pivot(index='driverCode', columns='round', values='points')

In [6]:
results['6']=results.sum(axis=1)
print(results)

round          1     2     3     4     5     6     7     8      6
driverCode                                                       
ALB          0.0   0.0   0.0   0.0   0.0   0.0   0.0   2.0    2.0
ALO          2.0  10.0   4.0   8.0   7.0   2.0   0.0   0.0   33.0
BEA          NaN   6.0   NaN   NaN   NaN   NaN   NaN   NaN    6.0
BOT          0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0
GAS          0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0    1.0
HAM          6.0   2.0   0.0   2.0   9.0   8.0   8.0   7.0   42.0
HUL          0.0   1.0   2.0   0.0   1.0   2.0   0.0   0.0    6.0
LEC         12.0  16.0  19.0  12.0  17.0  22.0  15.0  25.0  138.0
MAG          0.0   0.0   1.0   0.0   0.0   0.0   0.0   0.0    1.0
NOR          8.0   4.0  15.0  10.0  21.0  25.0  18.0  12.0  113.0
OCO          0.0   0.0   0.0   0.0   0.0   1.0   0.0   0.0    1.0
PER         18.0  18.0  10.0  18.0  21.0  18.0   4.0   0.0  107.0
PIA          4.0  12.0  12.0   4.0   6.0   3.0  12.0  18.0   71.0
RIC       

In [7]:
# Rank by pts
results['total_points']=results.sum(axis=1)
results=results.sort_values(by='total_points', ascending=False)

results.drop(columns='total_points', inplace=True)
print(results)

round          1     2     3     4     5     6     7     8      6
driverCode                                                       
VER         26.0  25.0   0.0  26.0  33.0  26.0  25.0   8.0  169.0
LEC         12.0  16.0  19.0  12.0  17.0  22.0  15.0  25.0  138.0
NOR          8.0   4.0  15.0  10.0  21.0  25.0  18.0  12.0  113.0
SAI         15.0   NaN  25.0  15.0  14.0  14.0  10.0  15.0  108.0
PER         18.0  18.0  10.0  18.0  21.0  18.0   4.0   0.0  107.0
PIA          4.0  12.0  12.0   4.0   6.0   3.0  12.0  18.0   71.0
RUS         10.0   8.0   0.0   6.0   9.0   4.0   7.0  10.0   54.0
HAM          6.0   2.0   0.0   2.0   9.0   8.0   8.0   7.0   42.0
ALO          2.0  10.0   4.0   8.0   7.0   2.0   0.0   0.0   33.0
TSU          0.0   0.0   6.0   1.0   0.0   7.0   1.0   4.0   19.0
STR          1.0   0.0   8.0   0.0   0.0   0.0   2.0   0.0   11.0
BEA          NaN   6.0   NaN   NaN   NaN   NaN   NaN   NaN    6.0
HUL          0.0   1.0   2.0   0.0   1.0   2.0   0.0   0.0    6.0
RIC       

In [8]:
# plot
fig = px.imshow(
    results.drop(columns='6'),
    text_auto=True,
    aspect='auto',
    color_continuous_scale=[[0,    'rgb(198, 219, 239)'],  # Blue scale
                            [0.25, 'rgb(107, 174, 214)'],
                            [0.5,  'rgb(33,  113, 181)'],
                            [0.75, 'rgb(8,   81,  156)'],
                            [1,    'rgb(8,   48,  107)']],
    labels={'x':'Race',
            'y':'Driver',
            'color':'Points'}
)
fig2 = px.imshow(
    results[['6']].values,
    text_auto=True,
    aspect='auto',
    color_continuous_scale='Reds',
    labels={'x':'Race',
            'y':'Driver',
            'color':'Points'}
)
# Update the layout of the second figure to adjust the position and size
""" fig2.update_layout(xaxis=dict(side='top', visible=False),
                   yaxis=dict(side='right', title='Driver'),
                   coloraxis_colorbar=dict(yanchor='top', y=1, ypad=20)) """

# Combine the two figures into one
fig = go.Figure(data=[fig.data[0], fig2.data[0]], layout=fig.layout)

fig.update_yaxes(title_text='')
fig.update_yaxes(tickmode='linear', showgrid=True, gridwidth=1, gridcolor='lightgrey', showline=False, tickson='boundaries')

fig.update_xaxes(title_text='')
fig.update_xaxes(showgrid=False, showline=False)
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')     # White background
fig.update_layout(coloraxis_showscale=False)        # Remove legend
fig.update_layout(xaxis=dict(side='top'))           # x-axis on top
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))  # Remove border margins
fig
show(fig)

In [9]:
fastf1.Cache.clear_cache()